In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Bibliotecas

In [3]:
!pip install GPy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 959 kB 5.4 MB/s 
     |████████████████████████████████| 71 kB 6.2 MB/s 
  Created wheel for GPy: filename=GPy-1.10.0-cp37-cp37m-linux_x86_64.whl size=2565156 sha256=06d7efb9220a87558a0fec43cca2f71f87057310b27c47ce2010823c794f262c
  Stored in directory: /root/.cache/pip/wheels/f7/18/28/dd1ce0192a81b71a3b086fd952511d088b21e8359ea496860a
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102566 sha256=482fa45660adc4b451fdded288e32eb6947ed64392eb608102bbe3ade7e2e02e
  Stored in directory: /root/.cache/pip/wheels/c8/95/f5/ce28482da28162e6028c4b3a32c41d147395825b3cd62bc810
Successfully built GPy paramz


In [4]:
!pip install memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=974369872790e62ca44e99a878bd07f1701a3dcc03cad4e715af6b30843b4442
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from math import ceil, floor, exp, sqrt, log
import sys
import traceback
import GPy
from GPy.kern import *
import random
import sys
import time
import os
import matplotlib.pyplot as plt
import logging
import csv
from memory_profiler import memory_usage

from datetime import date
from GPy.core import Mapping

ModuleNotFoundError: ignored

In [ ]:
import ipdb

In [ ]:
# create logger for the application
logger = logging.getLogger('DGPM Logger')

ch = logging.StreamHandler()

# create formatter and add it to the handlers
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)


logger.addHandler(ch)
logger.setLevel(logging.DEBUG)

Função para leitura dos dados

In [ ]:
def read_data():
    os.chdir('/content/drive/MyDrive/DGP_YCF_py/data')
    fp = 'Src_2_2006_to_Present_Treasury_Data.csv'
    df = pd.read_csv(fp)
    ref_date = date(2012, 12, 31)
    df["Date"] = pd.to_datetime(df["Date"])
    return df

In [ ]:
read_data()

,Date,Mo_1,Mo_3,Mo_6,Yr_1,Yr_2,Yr_3,Yr_5,Yr_7,Yr_10,Yr_20,Yr_30
0,2006-02-09,4.32,4.52,4.67,4.66,4.66,4.62,4.55,4.55,4.54,4.72,4.51
1,2006-02-10,4.36,4.53,4.70,4.70,4.69,4.67,4.59,4.59,4.59,4.76,4.55
2,2006-02-13,4.38,4.55,4.71,4.70,4.68,4.66,4.58,4.58,4.58,4.76,4.56
3,2006-02-14,4.42,4.55,4.72,4.71,4.69,4.68,4.61,4.61,4.62,4.80,4.60
4,2006-02-15,4.39,4.55,4.70,4.70,4.71,4.68,4.60,4.60,4.61,4.78,4.58
...,...,...,...,...,...,...,...,...,...,...,...,...
2753,2017-02-13,0.50,0.52,0.63,0.82,1.20,1.48,1.92,2.24,2.43,2.77,3.03
2754,2017-02-14,0.51,0.54,0.66,0.84,1.25,1.53,1.98,2.29,2.47,2.81,3.07
2755,2017-02-15,0.53,0.54,0.67,0.86,1.27,1.57,2.01,2.33,2.51,2.84,3.09
2756,2017-02-16,0.51,0.53,0.66,0.82,1.22,1.50,1.95,2.26,2.45,2.80,3.05


In [ ]:
global error_df, perf_df, X, pred_mat, act_mat, rmse_df
df = read_data()
date_init = '2017-02-05'
df = df.loc[df["Date"] > date_init]
df = df.reset_index(drop=True)
dates = df["Date"]
miy = 12.0
mts = [1.0, 3.0, 6.0, 12.0, 24.0, 36.0, 60.0, 84.0, 120.0, 240.0, 360.0]
X = [t/miy for t in mts]
X = np.array(X)
X = np.reshape(X, (X.shape[0], 1))
N = df.shape[0]
pred_rows = N - 2

In [ ]:
error_mat = np.ones((pred_rows, 11), dtype=np.float64)
pred_mat = np.ones((pred_rows, 11), dtype=np.float64)
act_mat = np.ones((pred_rows, 11), dtype=np.float64)
rmse_mat = np.ones((1, 11), dtype=np.float64)
date_list = list()

In [ ]:
logger.info("Dataset has " + str(N) + " rows")
row_index = 0
mf = Mapping(1, 1)
date_list = list()

2022-05-27 23:23:12,047 - DGPM Logger - INFO - Dataset has 10 rows


In [ ]:
while row_index < (N-2):  # (N-2) is because we don't need to process last record
    dfr = df.loc[row_index, :]
    req_cols = filter(lambda v: v not in ["Date", "key"], df.columns)
    Y = dfr[req_cols]
    Y = np.reshape(Y.values, (Y.shape[0], 1))
    
    # log progress - update progress every 100 days
    if row_index % 100 == 0:
        logger.debug("Day %d processed..." % row_index)
    
    if row_index == 0:
        k = RBF(input_dim=1)
        m = GPy.models.GPRegression(X, Y, kernel=k)

    else:
        # The following block corresponds to the update
        # of Kalman filter
        req_cols = filter(lambda v: v not in ["Date", "key"], df.columns)
        act = df.loc[row_index, req_cols]
        act = np.reshape(act.values, (act.shape[0], 1))
        # post_pred is the prediction for the previous day
        residual = act - post_pred
        # updated estimate
        X_star = X
        t1 = m.kern.K(X_star, X)
        t2 = np.linalg.inv(m.kern.K(X, X) + m['Gaussian_noise.variance'] * np.eye(X.shape[0]))
        t3 = t1.dot(t2)
        t4 = t3.dot(residual)
        um = post_pred + t4
        um = um.astype('float64')

        def the_mf(X):
            return um
        mf.f = the_mf
        mf.update_gradients = lambda a, b: None
        k = RBF(input_dim=1)

        lik = GPy.likelihoods.Gaussian()
        m = GPy.core.GP(X, Y, kernel=k, likelihood=lik, mean_function=mf)

    m.constrain_positive('*.*')
    m.optimize()
    
    post_pred = m.predict(X)[0]
    row_index = row_index + 1
    date_list.append(dates[row_index])
    # prediction for the next day, the index in pred mat lags one behind
    # the index for dates
    estimates = post_pred.ravel()
    pred_mat[(row_index - 1), :] = post_pred.ravel()
    req_cols = filter(lambda v: v not in ["Date", "key"], df.columns)
    act = df.loc[row_index, req_cols]
    error_mat[(row_index - 1), :] = (act - estimates)*(act - estimates)
    #ipdb.set_trace()
    #m.plot()

2022-05-27 23:23:13,604 - DGPM Logger - DEBUG - Day 0 processed...
reconstraining parameters GP_regression
reconstraining parameters gp


In [ ]:
error_df = pd.DataFrame(error_mat)
req_cols = filter(lambda v: v not in ["Date", "key"], df.columns)
error_df.columns = req_cols

perf_df = pd.DataFrame()
perf_df["Term"] = X.ravel()
me = error_df.apply(np.mean, axis=0)
perf_df["Mean.Error"] = list(me)
sd = error_df.apply(np.std, axis=0)
perf_df["SD"] = list(sd)

In [ ]:
# compute the rmse for each term
N_preds = N - 2
for c in range(error_df.shape[1]):
    se = error_df.iloc[:, c]
    rmse_mat[0, c] = sqrt(np.sum(se)/N_preds)

rmse_df = pd.DataFrame(rmse_mat)
req_cols = filter(lambda v: v not in ["Date", "key"], df.columns)
rmse_df.columns = req_cols

In [ ]:
os.chdir('/content/drive/MyDrive/DGP_YCF_py/output')
fp = "GP_Results.csv"
perf_df.to_csv(fp, index=False, header=True)
fp = "rmse_results_DGP" + ".csv"
rmse_df.to_csv(fp, index=False, header=True)
error_df["Date"] = np.array(date_list)
fp = "10Y_GP_sq_term_errors.csv"
error_df.to_csv(fp, index=False, header=True)

In [ ]:
with open("day_1_sample.csv", 'a') as d1sample:
    writer = csv.writer(d1sample)
    line_to_write = pred_mat[0, :].tolist()
    line_to_write.append("GP")
    writer.writerow(line_to_write)

In [ ]:
with open("day_500_sample.csv", 'a') as d500sample:
    writer = csv.writer(d500sample)
    line_to_write = pred_mat[500, :].tolist()
    line_to_write.append("GP")
    writer.writerow(line_to_write)

In [ ]:
 with open("day_1000_sample.csv", 'a') as d1000sample:
    writer = csv.writer(d1000sample)
    line_to_write = pred_mat[1000, :].tolist()
    line_to_write.append("GP")
    writer.writerow(line_to_write)

In [ ]:
  with open("day_1500_sample.csv", 'a') as d1500sample:
    writer = csv.writer(d1500sample)
    line_to_write = pred_mat[1500, :].tolist()
    line_to_write.append("GP")
    writer.writerow(line_to_write)

In [ ]:
with open("day_2000_sample.csv", 'a') as d2000sample:
    writer = csv.writer(d2000sample)
    line_to_write = pred_mat[2000, :].tolist()
    line_to_write.append("GP")
    writer.writerow(line_to_write)

In [ ]:
logger.info("Done!")

In [ ]:
df = read_data()
k = 100
miy = 12.0
mts = [1.0, 3.0, 6.0, 12.0, 24.0, 36.0, 60.0, 84.0, 120.0, 240.0, 360.0]
X = [t/miy for t in mts]
X = np.array(X)
X = np.reshape(X, (X.shape[0], 1))
req_cols = filter(lambda v: v not in ["Date", "key"], df.columns)
Y = df.loc[k, req_cols]
Y = np.reshape(Y.values, (Y.shape[0], 1))
k = RBF(input_dim=1)
m = GPy.models.GPRegression(X, Y, k)
m.optimize()
m.plot()